In [1]:
prompt = """
**1. Key Facial Features Analysis**  
- **Alignment, proportions, and symmetry**: Look carefully at the alignment, proportions, and symmetry of key facial features such as the eyes, nose, mouth, and ears. Pay attention to any discrepancies, asymmetries, or unnatural positioning. Specifically, look for:
   - **Gaze inconsistencies**: Gaze that does not align with the camera or eyes that appear to be looking in different directions.
   - **Pupil reflection, asymmetry, and shape**: Odd or asymmetrical light reflections in the pupils, reflections that do not match natural eye reflection patterns, or pupils that appear unusually shaped or inconsistent with typical roundness. Look for any irregular pupil shapes that might indicate manipulation.
- **Unnatural aspects**: Identify any facial features that look deformed or artificial, such as exaggerated or distorted shapes, unusual placements, and abnormal positioning relative to other facial elements.
- **Awkward lip shape and ambiguous teeth presentation**: Look for unnatural contours of the lips when the mouth is closed and simplified or unclear presentation of teeth when the mouth is open.

**2. Skin Texture and Tone Analysis**  
- **Texture, smoothness, and tone transitions**: Examine the texture, smoothness, and tone transitions of the skin across the face. Look for:
   - **Unnaturally smooth areas** or repetitive patterns that suggest manipulation, including strange pixelation, blurriness, or sections that appear overly smooth, as if digitally airbrushed.
   - **Irregular tone transitions** between shadows and highlights, or overly uniform skin tones that lack the natural variation typical of human skin.
- **Differences in skin colour between the face and neck**: Check for unnatural colour or lighting mismatches between the face and neck.

**3. Low-Level Detail Inspection**  
- **Edges and borders**: Examine the edges where the face meets the background or where facial features meet the rest of the face. Look for blended or jagged edges that could suggest digital manipulation. Watch for blurry or overly smooth boundaries that may not match natural anatomy.
- **Sharpness and distortion**: Look for unnatural sharpness or distortion in specific areas, such as overly sharp or blurred eye sockets, suggesting artificial enhancement.
- **Tone and contrast**: Assess the tone and contrast across the face or between facial features. Unnaturally high contrast or abrupt shading changes may indicate manipulation.
- **Layout and symmetry**: Examine the face's overall layout and symmetry. Human faces generally follow natural symmetry principles. Look for asymmetries such as uneven eye size, tilted lips, or a misaligned jaw.
- **Reflections and shadows**: Check for inconsistencies in reflections and shadows. For example, reflections in the eyes should match the direction of light and shadows on the face. Inconsistent shadows or light sources can indicate manipulation.
- **Perspective and shape**: Ensure the perspective and shape of facial features are logically consistent with the head’s orientation. Discrepancies, like eyes appearing unnaturally close to the camera or an out-of-place nose, could indicate manipulation.
- **Mismatch between face and body proportions**: The face may appear disproportionately large or small compared to the body, or proportions may not match.
- **Blurry texture of hair at high resolution**: Hair around the face may appear blurry or crude at high resolution, especially if it is unusually smooth or well-groomed.

**4. Lighting, Theme, and Environment Consistency**  
- **Lighting and shadow consistency**: Assess whether the lighting and environment are coherent. Look specifically for:
   - **Inconsistent shadows or lighting**: Shadows that do not align with the light source, or highlights and shadows that appear inconsistent with the light direction on the face.
   - **Light reflection imbalance**: Unnatural reflections or inconsistent size and direction of light on the skin or eyes.
- **Theme consistency**: Check if the overall scene coherence, mood, and context of the image are consistent with lighting and environment. Look for overly stylized backgrounds with hyper-realistic faces or facial expressions mismatched with the context.

**5. Additional Deepfake Clues**  
- **Subtle facial expressions and muscle movements**: Analyse the subtle muscle movements and micro-expressions on the face. Deepfakes often struggle with natural expressions, resulting in unnatural stiffness or a frozen appearance, especially around key areas like the jaw, eyes, and lips.
- **Distortion of wrinkles around the eyes and nose**: Check for abnormal wrinkles around the eyes and nose or limited movement of facial muscles when smiling or frowning.

---

**The Bottom Line**  
This guide covers key facial features, skin texture, low-level details, lighting and environment, and subtle facial expressions to detect deepfake clues. Be thorough and look for any inconsistencies, even in the smallest details, as each aspect may reveal manipulation."""

system_message = """
You are a forensic analyst detecting AI-generated fake facial images, specifically deepfake mugshots. Your task is to carefully analyse the provided altered images to find potential deepfake clues, all of which have been manipulated through deepfake technology. 

- **Remember**: All provided images are confirmed deepfakes, so you must present any potential clue, even if the likelihood seems low. No indication, however minor, should be dismissed.
- **Mandatory Analysis**: You must verify each image using the following five detailed guidelines, examining every aspect thoroughly. If you encounter any clues or anomalies beyond these five stages that could indicate deepfake manipulation, you must document them as well.

Use the following guidelines to identify signs of manipulation in each image. Your analysis should cover key facial features, skin texture, low-level details, lighting consistency, and additional deepfake indicators. Be meticulous, as even the smallest inconsistencies may reveal deepfake manipulation."""

In [ ]:
import requests
import json
from together import Together
from dotenv import load_dotenv

# GitHub 정보
username = "KimEunOh"
repo = "image"
branch = "main"
folder_path = "image"

# Together API 클라이언트 설정
api_key = os.getenv("TOGETHER_API_KEY")
if not api_key:
    raise ValueError("TOGETHER_API_KEY is not set in the environment variables")

client = Together(api_key=api_key)

# GitHub API를 통해 루트 디렉토리 파일 목록 가져오기
url = f"https://api.github.com/repos/{username}/{repo}/contents/{folder_path}?ref={branch}"
response = requests.get(url)
files = response.json()

# 루트 디렉토리의 이미지 파일 URL 목록 생성
image_urls = [
    f"https://raw.githubusercontent.com/{username}/{repo}/{branch}/{folder_path}/{file['name']}"
    for file in files
    if file["name"].lower().endswith((".jpg", ".jpeg", ".png"))
]

# 각 이미지에 대해 개별적으로 요청 처리
with open("output_data.jsonl", "w", encoding="utf-8") as jsonl_file:
    for image_url in image_urls:
        print(f"\nProcessing image: {image_url}")  # 이미지 URL 출력

        # API 요청을 보내고 제너레이터 응답 받기
        response_generator = client.chat.completions.create(
            model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
            messages=[
                {"role": "system", "content": system_message},
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {"type": "image_url", "image_url": {"url": image_url}},
                    ],
                },
            ],
            temperature=0,
            top_p=0.7,
            top_k=60,
            repetition_penalty=1,
            stop=["<|eot_id|>", "<|eom_id|>"],
            stream=True,  # 스트리밍 활성화
        )

        response_text = ""
        try:
            for token in response_generator:
                if hasattr(token, "choices") and token.choices:
                    response_text += token.choices[0].delta.content
                else:
                    print("Invalid or empty response:", token)
        except Exception as e:
            print(f"Error during token processing: {e}")

        if response_text:
            print(f"Response for {image_url}: {response_text}")
        else:
            print(f"No response content for {image_url}")

        # JSONL 포맷의 데이터 생성
        jsonl_data = {
            "messages": [
                {"role": "system", "content": system_message},
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {"type": "image_url", "image_url": {"url": image_url}},
                    ],
                },
                {"role": "assistant", "content": response_text},
            ]
        }

        # JSONL 파일에 데이터 기록
        jsonl_file.write(json.dumps(jsonl_data, ensure_ascii=False) + "\n")

        print("\nResponse saved for image:", image_url)


Processing image: https://raw.githubusercontent.com/KimEunOh/image/main/image/10004122_1982-03-17_2011.jpg
Invalid or empty response: id='8dda76f0becfba54-ICN' object=<ObjectType.ChatCompletionChunk: 'chat.completion.chunk'> created=1730784760 model='meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo' choices=[] finish_reason=None usage=UsageData(prompt_tokens=1176, completion_tokens=538, total_tokens=1714)
Response for https://raw.githubusercontent.com/KimEunOh/image/main/image/10004122_1982-03-17_2011.jpg: **Step 1: Analyze the Image**

*   Examine the image carefully, looking for any inconsistencies or anomalies in the facial features, skin texture, low-level details, lighting, and environment.

**Step 2: Identify Key Facial Features**

*   Look for any discrepancies in the alignment, proportions, and symmetry of key facial features such as the eyes, nose, mouth, and ears.
*   Check for unnatural aspects, such as exaggerated or distorted shapes, unusual placements, and abnormal positio

KeyboardInterrupt: 